# Can a card's attributes be used to predict its price?

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import random

In [26]:
# random.seed(42)

# all_sets = pd.read_csv("set_info.csv")
# major_sets = all_sets[all_sets["set_size"] > 100]
# sample = major_sets.sample(1)
# print(major_sets)
# incomplete = major_sets.drop([3,5,7,11,12,21,22,23,25,26,28,35,44,47,58,69,81,87])

main_sets = pd.read_csv(".\data\set_overview.csv")
print(len(main_sets))
# main_sets = main_sets[73:]
# main_sets
print(main_sets)
pd.read_csv(".\data\latest_printings.csv")

108
                                set_name set_id  set_size  set_year
0                          Core Set 2020    m20       344      2019
1                          Core Set 2019    m19       314      2018
2                          Magic Origins    ori       288      2015
3                             Magic 2015    m15       284      2014
4                             Magic 2014    m14       249      2013
5                             Magic 2013    m13       249      2012
6                             Magic 2012    m12       249      2011
7                             Magic 2011    m11       249      2010
8                             Magic 2010    m10       249      2009
9                          Tenth Edition    10e       383      2007
10                         Ninth Edition    9ed       359      2005
11                        Eighth Edition    8ed       357      2003
12                       Seventh Edition    7ed       350      2001
13                 Classic Sixth Edition    

,name,supertype,MAX(set_year),rarity,cmc,color,modern_legality,standard_legality,commander_legality,vintage_legality,legacy_legality,reprint,price,is_legendary,edhrec
0,Aerial Assault,Sorcery,2019,Common,3,White,Legal,Legal,Legal,Legal,Legal,0,0.06,0,184
1,Ajani Strength of the Pride,Planeswalker,2019,Mythic Rare,4,White,Legal,Legal,Legal,Legal,Legal,0,12.42,0,2164
2,Ancestral Blade,Artifact,2019,Uncommon,2,White,Legal,Legal,Legal,Legal,Legal,0,0.11,0,392
3,Angel of Vitality,Creature,2019,Uncommon,3,White,Legal,Legal,Legal,Legal,Legal,0,0.26,0,1720
4,Angelic Gift,Enchantment,2019,Common,2,White,Legal,Legal,Legal,Legal,Legal,0,0.09,0,946
5,Apostle of Purifying Light,Creature,2019,Uncommon,2,White,Legal,Legal,Legal,Legal,Legal,0,0.16,0,147
6,Battalion Foot Soldier,Creature,2019,Common,3,White,Legal,Legal,Legal,Legal,Legal,0,0.07,0,3
7,Bishop of Wings,Creature,2019,Rare,2,White,Legal,Legal,Legal,Legal,Legal,0,0.68,0,990
8,Brought Back,Instant,2019,Rare,2,White,Legal,Legal,Legal,Legal,Legal,0,0.64,0,1870
9,Cavalier of Dawn,Creature,2019,Mythic Rare,5,White,Legal,Legal,Legal,Legal,Legal,0,2.55,0,2143


In [13]:
for set_, size in zip(main_sets["set_id"], main_sets["set_size"]):
    names = []
    supertypes = []
    subtypes = []
    sets = []
    rarities = []
    cmcs = []
    colors = []
    modern_legalities = []
    standard_legalities = []
    commander_legalities = []
    vintage_legalities = []
    legacy_legalities = []
    reprints = []
    artists = []
    prices = []
    are_legendary = []
    for card_num in range(size):
        print(set_, card_num+1)
        try:
            response = requests.get('https://scryfall.com/card/{}/{}/'.format(set_, str(card_num+1)))
            html = BeautifulSoup(response.content, 'html.parser')

            card_title = html.find("h1", class_ = "card-text-title").text

            try:
                card_has_suspend = "Suspend" in html.find("div", class_ = "card-text-oracle").text.strip()
            except AttributeError:
                card_has_suspend = False        
            if card_has_suspend == True:
                continue
            

            card_name = card_title.strip().split("\n")[0]
            card_name = re.sub(",", "", card_name)
            if card_name == "Evermind":
                continue
            names.append(card_name)

            card_type = html.find("p", class_ = "card-text-type-line").text.strip()
            if "—" in card_type:
                card_types = card_type.split("—")
                card_supertype = card_types[0].strip()
                card_subtype = card_types[1].strip()
                subtypes.append(card_subtype)
            elif "—" not in card_type:
                card_supertype = card_type
                card_subtype = "None"   
                subtypes.append(card_subtype)

            if "Legendary" in card_supertype:
                is_legendary = True
                card_supertype = re.sub("Legendary ", "", card_supertype)
                are_legendary.append(is_legendary)
                supertypes.append(card_supertype)
            else:
                is_legendary = False
                are_legendary.append(is_legendary)
                supertypes.append(card_supertype)

            card_artist = html.find("p", class_ = "card-text-artist").text.strip().split("\n")[1].strip()
            artists.append(card_artist.encode('utf-8'))

            card_current_prints = html.find("div", class_ = "prints-current").text.strip().replace("\n", "").split("              ")
            card_set = card_current_prints[0]
            wrds = card_set.strip().split(" ")[0:-1]
            card_set = " ".join(wrds)
            sets.append(card_set)
            card_rarity = card_current_prints[1].split(" · ")[1]
            rarities.append(card_rarity)

            card_legalities = []
            for j in html.find_all("div", class_ = "card-legality-item"):
                card_legalities.append(j.text.split())
            for l in card_legalities:
                if l[0] == "Standard":
                    if (l[1] == "Not"):
                        card_standard_legal = "Not Legal"
                    elif l[1] == "Banned":
                        card_standard_legal = "Banned"
                    elif l[1] == "Legal":
                        card_standard_legal = "Legal"
                if l[0] == "Modern":
                    if (l[1] == "Not"):
                        card_modern_legal = "Not Legal"
                    elif l[1] == "Banned":
                        card_modern_legal = "Banned"
                    elif l[1] == "Legal":
                        card_modern_legal = "Legal"
                if l[0] == "Commander":
                    if (l[1] == "Not"):
                        card_commander_legal = "Not Legal"
                    elif l[1] == "Banned":
                        card_commander_legal = "Banned"
                    elif l[1] == "Legal":
                        card_commander_legal = "Legal"
                if l[0] == "Vintage":
                    if (l[1] == "Not"):
                        card_vintage_legal = "Not Legal"
                    elif l[1] == "Banned":
                        card_vintage_legal = "Banned"
                    elif l[1] == "Legal":
                        card_vintage_legal = "Legal"
                    elif l[1] == "Restrict.":
                        card_vintage_legal = "Restrict."
                if l[0] == "Legacy":
                    if (l[1] == "Not"):
                        card_legacy_legal = "Not Legal"
                    elif l[1] == "Banned":
                        card_legacy_legal = "Banned"
                    elif l[1] == "Legal":
                        card_legacy_legal = "Legal"
            standard_legalities.append(card_standard_legal)
            modern_legalities.append(card_modern_legal)
            commander_legalities.append(card_commander_legal)
            vintage_legalities.append(card_vintage_legal)
            legacy_legalities.append(card_legacy_legal)

            for k in html.find_all("tr", class_ = "current"):
                if len(re.findall("\d*\.\d{2}", k.text.strip())) != 0:
                    card_prices = re.findall("\d*\.\d{2}", k.text.strip())
                    card_price = float(card_prices[0])
            prices.append(card_price)

            l = []
            for z in html.select("div[class = prints] > table[class=prints-table]"):
                l.append(z.text)
            for n in l:
                sections = re.sub("  {1,}", " ", n.replace("\n", ""))
                if "Prints" in sections:
                    if len(sections.split()) > 6:
                        card_is_reprint = True
                    else:
                        card_is_reprint = False
                else:
                    continue       
            reprints.append(card_is_reprint)

            if (card_has_suspend == False) & ("Land" not in card_supertype):
                card_casting_cost = re.findall("[A-Z1-9]", card_title.strip().split("\n")[1].strip())
                try: 
                    if len(card_casting_cost) == 0:
                        card_cmc = 0
                    else:
                        generic_mana_count = int(card_casting_cost[0])
                        card_cmc = generic_mana_count + len(card_casting_cost[1:])
                except ValueError:
                    card_cmc = len(card_casting_cost)

                card_color_list = []
                if "W" in card_casting_cost:
                    card_color_list.append("White")
                if "U" in card_casting_cost:
                    card_color_list.append("Blue")
                if "B" in card_casting_cost:
                    card_color_list.append("Black")
                if "R" in card_casting_cost:
                    card_color_list.append("Red")
                if "G" in card_casting_cost:
                    card_color_list.append("Green")
                if len(card_color_list) == 1:
                    card_color = card_color_list[0]
                if len(card_color_list) > 1:
                    card_color = "Gold"
                if len(card_color_list) == 0:
                    card_color = "Colorless"
                cmcs.append(card_cmc)
                colors.append(card_color)

            elif (card_has_suspend == False) & (("Land" in card_supertype)):
                card_cmc = 0
                card_color = "Colorless"
                cmcs.append(card_cmc)
                colors.append(card_color)
        except AttributeError as e:
            print(e)
            continue

    data = {"names": names, "supertypes": supertypes, "subtypes": subtypes, "set": sets, "rarity": rarities, "cmc": cmcs, "color": colors, "modern legality": modern_legalities, "standard legality": standard_legalities, "commander legality": commander_legalities, "vintage legality": vintage_legalities, "legacy legality": legacy_legalities, "reprint": reprints, "artist": artists, "price": prices, "is_legendary": are_legendary}
    df = pd.DataFrame(data)
    df.to_csv(".\data\main_sets\{}_info.csv".format(set_), index = False)

m20 1
m20 2
m20 3
m20 4
m20 5
m20 6
m20 7
m20 8
m20 9
m20 10
m20 11
m20 12
m20 13
m20 14
m20 15
m20 16
m20 17
m20 18
m20 19
m20 20
m20 21
m20 22
m20 23
m20 24
m20 25
m20 26
m20 27
m20 28
m20 29
m20 30
m20 31
m20 32
m20 33
m20 34
m20 35
m20 36
m20 37
m20 38
m20 39
m20 40
m20 41
m20 42
m20 43
m20 44
m20 45
m20 46
m20 47
m20 48
m20 49
m20 50
m20 51
m20 52
m20 53
m20 54
m20 55
m20 56
m20 57
m20 58
m20 59
m20 60
m20 61
m20 62
m20 63
m20 64
m20 65
m20 66
m20 67
m20 68
m20 69
m20 70
m20 71
m20 72
m20 73
m20 74
m20 75
m20 76
m20 77
m20 78
m20 79
m20 80
m20 81
m20 82
m20 83
m20 84
m20 85
m20 86
m20 87
m20 88
m20 89
m20 90
m20 91
m20 92
m20 93
m20 94
m20 95
m20 96
m20 97
m20 98
m20 99
m20 100
m20 101
m20 102
m20 103
m20 104
m20 105
m20 106
m20 107
m20 108
m20 109
m20 110
m20 111
m20 112
m20 113
m20 114
m20 115
m20 116
m20 117
m20 118
m20 119
m20 120
m20 121
m20 122
m20 123
m20 124
m20 125
m20 126
m20 127
m20 128
m20 129
m20 130
m20 131
m20 132
m20 133
m20 134
m20 135
m20 136
m20 137
m20 138
m20 

m15 134
m15 135
m15 136
m15 137
m15 138
m15 139
m15 140
m15 141
m15 142
m15 143
m15 144
m15 145
m15 146
m15 147
m15 148
m15 149
m15 150
m15 151
m15 152
m15 153
m15 154
m15 155
m15 156
m15 157
m15 158
m15 159
m15 160
m15 161
m15 162
m15 163
m15 164
m15 165
m15 166
m15 167
m15 168
m15 169
m15 170
m15 171
m15 172
m15 173
m15 174
m15 175
m15 176
m15 177
m15 178
m15 179
m15 180
m15 181
m15 182
m15 183
m15 184
m15 185
m15 186
m15 187
m15 188
m15 189
m15 190
m15 191
m15 192
m15 193
m15 194
m15 195
m15 196
m15 197
m15 198
m15 199
m15 200
m15 201
m15 202
m15 203
m15 204
m15 205
m15 206
m15 207
m15 208
m15 209
m15 210
m15 211
m15 212
m15 213
m15 214
m15 215
m15 216
m15 217
m15 218
m15 219
m15 220
m15 221
m15 222
m15 223
m15 224
m15 225
m15 226
m15 227
m15 228
m15 229
m15 230
m15 231
m15 232
m15 233
m15 234
m15 235
m15 236
m15 237
m15 238
m15 239
m15 240
m15 241
m15 242
m15 243
m15 244
m15 245
m15 246
m15 247
m15 248
m15 249
m15 250
m15 251
m15 252
m15 253
m15 254
m15 255
m15 256
m15 257
m15 258


m11 182
m11 183
m11 184
m11 185
m11 186
m11 187
m11 188
m11 189
m11 190
m11 191
m11 192
m11 193
m11 194
m11 195
m11 196
m11 197
m11 198
m11 199
m11 200
m11 201
m11 202
m11 203
m11 204
m11 205
m11 206
m11 207
m11 208
m11 209
m11 210
m11 211
m11 212
m11 213
m11 214
m11 215
m11 216
m11 217
m11 218
m11 219
m11 220
m11 221
m11 222
m11 223
m11 224
m11 225
m11 226
m11 227
m11 228
m11 229
m11 230
m11 231
m11 232
m11 233
m11 234
m11 235
m11 236
m11 237
m11 238
m11 239
m11 240
m11 241
m11 242
m11 243
m11 244
m11 245
m11 246
m11 247
m11 248
m11 249
m10 1
m10 2
m10 3
m10 4
m10 5
m10 6
m10 7
m10 8
m10 9
m10 10
m10 11
m10 12
m10 13
m10 14
m10 15
m10 16
m10 17
m10 18
m10 19
m10 20
m10 21
m10 22
m10 23
m10 24
m10 25
m10 26
m10 27
m10 28
m10 29
m10 30
m10 31
m10 32
m10 33
m10 34
m10 35
m10 36
m10 37
m10 38
m10 39
m10 40
m10 41
m10 42
m10 43
m10 44
m10 45
m10 46
m10 47
m10 48
m10 49
m10 50
m10 51
m10 52
m10 53
m10 54
m10 55
m10 56
m10 57
m10 58
m10 59
m10 60
m10 61
m10 62
m10 63
m10 64
m10 65
m10 66
m10

'NoneType' object has no attribute 'text'
9ed 355
'NoneType' object has no attribute 'text'
9ed 356
'NoneType' object has no attribute 'text'
9ed 357
'NoneType' object has no attribute 'text'
9ed 358
'NoneType' object has no attribute 'text'
9ed 359
'NoneType' object has no attribute 'text'
8ed 1
8ed 2
8ed 3
8ed 4
8ed 5
8ed 6
8ed 7
8ed 8
8ed 9
8ed 10
8ed 11
8ed 12
8ed 13
8ed 14
8ed 15
8ed 16
8ed 17
8ed 18
8ed 19
8ed 20
8ed 21
8ed 22
8ed 23
8ed 24
8ed 25
8ed 26
8ed 27
8ed 28
8ed 29
8ed 30
8ed 31
8ed 32
8ed 33
8ed 34
8ed 35
8ed 36
8ed 37
8ed 38
8ed 39
8ed 40
8ed 41
8ed 42
8ed 43
8ed 44
8ed 45
8ed 46
8ed 47
8ed 48
8ed 49
8ed 50
8ed 51
8ed 52
8ed 53
8ed 54
8ed 55
8ed 56
8ed 57
8ed 58
8ed 59
8ed 60
8ed 61
8ed 62
8ed 63
8ed 64
8ed 65
8ed 66
8ed 67
8ed 68
8ed 69
8ed 70
8ed 71
8ed 72
8ed 73
8ed 74
8ed 75
8ed 76
8ed 77
8ed 78
8ed 79
8ed 80
8ed 81
8ed 82
8ed 83
8ed 84
8ed 85
8ed 86
8ed 87
8ed 88
8ed 89
8ed 90
8ed 91
8ed 92
8ed 93
8ed 94
8ed 95
8ed 96
8ed 97
8ed 98
8ed 99
8ed 100
8ed 101
8ed 102


6ed 286
6ed 287
6ed 288
6ed 289
6ed 290
6ed 291
6ed 292
6ed 293
6ed 294
6ed 295
6ed 296
6ed 297
6ed 298
6ed 299
6ed 300
6ed 301
6ed 302
6ed 303
6ed 304
6ed 305
6ed 306
6ed 307
6ed 308
6ed 309
6ed 310
6ed 311
6ed 312
6ed 313
6ed 314
6ed 315
6ed 316
6ed 317
6ed 318
6ed 319
6ed 320
6ed 321
6ed 322
6ed 323
6ed 324
6ed 325
6ed 326
6ed 327
6ed 328
6ed 329
6ed 330
6ed 331
6ed 332
6ed 333
6ed 334
6ed 335
6ed 336
6ed 337
6ed 338
6ed 339
6ed 340
6ed 341
6ed 342
6ed 343
6ed 344
6ed 345
6ed 346
6ed 347
6ed 348
6ed 349
6ed 350
5ed 1
5ed 2
5ed 3
5ed 4
5ed 5
5ed 6
5ed 7
5ed 8
5ed 9
5ed 10
5ed 11
5ed 12
5ed 13
5ed 14
5ed 15
5ed 16
5ed 17
5ed 18
5ed 19
5ed 20
5ed 21
5ed 22
5ed 23
5ed 24
5ed 25
5ed 26
5ed 27
5ed 28
5ed 29
5ed 30
5ed 31
5ed 32
5ed 33
5ed 34
5ed 35
5ed 36
5ed 37
5ed 38
5ed 39
5ed 40
5ed 41
5ed 42
5ed 43
5ed 44
5ed 45
5ed 46
5ed 47
5ed 48
5ed 49
5ed 50
5ed 51
5ed 52
5ed 53
5ed 54
5ed 55
5ed 56
5ed 57
5ed 58
5ed 59
5ed 60
5ed 61
5ed 62
5ed 63
5ed 64
5ed 65
5ed 66
5ed 67
5ed 68
5ed 69
5ed 70

...

usg 163
usg 164
usg 165
usg 166
usg 167
usg 168
usg 169
usg 170
usg 171
usg 172
usg 173
usg 174
usg 175
usg 176
usg 177
usg 178
usg 179
usg 180
usg 181
usg 182
usg 183
usg 184
usg 185
usg 186
usg 187
usg 188
usg 189
usg 190
usg 191
usg 192
usg 193
usg 194
usg 195
usg 196
usg 197
usg 198
usg 199
usg 200
usg 201
usg 202
usg 203
usg 204
usg 205
usg 206
usg 207
usg 208
usg 209
usg 210
usg 211
usg 212
usg 213
usg 214
usg 215
usg 216
usg 217
usg 218
usg 219
usg 220
usg 221
usg 222
usg 223
usg 224
usg 225
usg 226
usg 227
usg 228
usg 229
usg 230
usg 231
usg 232
usg 233
usg 234
usg 235
usg 236
usg 237
usg 238
usg 239
usg 240
usg 241
usg 242
usg 243
usg 244
usg 245
usg 246
usg 247
usg 248
usg 249
usg 250
usg 251
usg 252
usg 253
usg 254
usg 255
usg 256
usg 257
usg 258
usg 259
usg 260
usg 261
usg 262
usg 263
usg 264
usg 265
usg 266
usg 267
usg 268
usg 269
usg 270
usg 271
usg 272
usg 273
usg 274
usg 275
usg 276
usg 277
usg 278
usg 279
usg 280
usg 281
usg 282
usg 283
usg 284
usg 285
usg 286
usg 287


vis 102
vis 103
vis 104
vis 105
vis 106
vis 107
vis 108
vis 109
vis 110
vis 111
vis 112
vis 113
vis 114
vis 115
vis 116
vis 117
vis 118
vis 119
vis 120
vis 121
vis 122
vis 123
vis 124
vis 125
vis 126
vis 127
vis 128
vis 129
vis 130
vis 131
vis 132
vis 133
vis 134
vis 135
vis 136
vis 137
vis 138
vis 139
vis 140
vis 141
vis 142
vis 143
vis 144
vis 145
vis 146
vis 147
vis 148
vis 149
vis 150
vis 151
vis 152
vis 153
vis 154
vis 155
vis 156
vis 157
vis 158
vis 159
vis 160
vis 161
vis 162
vis 163
vis 164
vis 165
vis 166
vis 167
mir 1
mir 2
mir 3
mir 4
mir 5
mir 6
mir 7
mir 8
mir 9
mir 10
mir 11
mir 12
mir 13
mir 14
mir 15
mir 16
mir 17
mir 18
mir 19
mir 20
mir 21
mir 22
mir 23
mir 24
mir 25
mir 26
mir 27
mir 28
mir 29
mir 30
mir 31
mir 32
mir 33
mir 34
mir 35
mir 36
mir 37
mir 38
mir 39
mir 40
mir 41
mir 42
mir 43
mir 44
mir 45
mir 46
mir 47
mir 48
mir 49
mir 50
mir 51
mir 52
mir 53
mir 54
mir 55
mir 56
mir 57
mir 58
mir 59
mir 60
mir 61
mir 62
mir 63
mir 64
mir 65
mir 66
mir 67
mir 68
mir 6

'NoneType' object has no attribute 'text'
all 179
'NoneType' object has no attribute 'text'
all 180
'NoneType' object has no attribute 'text'
all 181
'NoneType' object has no attribute 'text'
all 182
'NoneType' object has no attribute 'text'
all 183
'NoneType' object has no attribute 'text'
all 184
'NoneType' object has no attribute 'text'
all 185
'NoneType' object has no attribute 'text'
all 186
'NoneType' object has no attribute 'text'
all 187
'NoneType' object has no attribute 'text'
all 188
'NoneType' object has no attribute 'text'
all 189
'NoneType' object has no attribute 'text'
all 190
'NoneType' object has no attribute 'text'
all 191
'NoneType' object has no attribute 'text'
all 192
'NoneType' object has no attribute 'text'
all 193
'NoneType' object has no attribute 'text'
all 194
'NoneType' object has no attribute 'text'
all 195
'NoneType' object has no attribute 'text'
all 196
'NoneType' object has no attribute 'text'
all 197
'NoneType' object has no attribute 'text'
all 198


'NoneType' object has no attribute 'text'
fem 42
'NoneType' object has no attribute 'text'
fem 43
fem 44
fem 45
fem 46
fem 47
fem 48
fem 49
'NoneType' object has no attribute 'text'
fem 50
fem 51
fem 52
fem 53
'NoneType' object has no attribute 'text'
fem 54
'NoneType' object has no attribute 'text'
fem 55
fem 56
'NoneType' object has no attribute 'text'
fem 57
fem 58
'NoneType' object has no attribute 'text'
fem 59
fem 60
fem 61
'NoneType' object has no attribute 'text'
fem 62
'NoneType' object has no attribute 'text'
fem 63
fem 64
fem 65
'NoneType' object has no attribute 'text'
fem 66
fem 67
'NoneType' object has no attribute 'text'
fem 68
'NoneType' object has no attribute 'text'
fem 69
fem 70
fem 71
'NoneType' object has no attribute 'text'
fem 72
'NoneType' object has no attribute 'text'
fem 73
fem 74
'NoneType' object has no attribute 'text'
fem 75
fem 76
fem 77
fem 78
fem 79
fem 80
'NoneType' object has no attribute 'text'
fem 81
fem 82
fem 83
fem 84
fem 85
fem 86
fem 87
fem 88

leg 265
leg 266
leg 267
leg 268
leg 269
leg 270
leg 271
leg 272
leg 273
leg 274
leg 275
leg 276
leg 277
leg 278
leg 279
leg 280
leg 281
leg 282
leg 283
leg 284
leg 285
leg 286
leg 287
leg 288
leg 289
leg 290
leg 291
leg 292
leg 293
leg 294
leg 295
leg 296
leg 297
leg 298
leg 299
leg 300
leg 301
leg 302
leg 303
leg 304
leg 305
leg 306
leg 307
leg 308
leg 309
leg 310
atq 1
atq 2
atq 3
atq 4
atq 5
atq 6
atq 7
atq 8
atq 9
atq 10
atq 11
atq 12
atq 13
atq 14
atq 15
atq 16
atq 17
atq 18
atq 19
atq 20
atq 21
atq 22
atq 23
atq 24
atq 25
atq 26
atq 27
atq 28
atq 29
atq 30
atq 31
atq 32
atq 33
atq 34
atq 35
atq 36
atq 37
atq 38
atq 39
atq 40
atq 41
atq 42
atq 43
atq 44
atq 45
atq 46
atq 47
atq 48
atq 49
atq 50
atq 51
atq 52
atq 53
atq 54
atq 55
atq 56
atq 57
atq 58
atq 59
atq 60
atq 61
atq 62
atq 63
atq 64
atq 65
atq 66
atq 67
atq 68
atq 69
atq 70
atq 71
atq 72
atq 73
atq 74
atq 75
atq 76
atq 77
atq 78
atq 79
atq 80
'NoneType' object has no attribute 'text'
atq 81
atq 82
'NoneType' object has no 

In [9]:
response = requests.get('https://scryfall.com/card/arn/76/library-of-alexandria')
html = BeautifulSoup(response.content, 'html.parser')

In [11]:
card_title = html.find("h1", class_ = "card-text-title").text

try:
    card_has_suspend = "Suspend" in html.find("div", class_ = "card-text-oracle").text.strip()
except AttributeError:
    card_has_suspend = False
# if card_has_suspend == True:
#     continue


card_name = card_title.strip().split("\n")[0]
print(card_name)

card_type = html.find("p", class_ = "card-text-type-line").text.strip()
if "—" in card_type:
    card_types = card_type.split("—")
    card_supertype = card_types[0].strip()
    card_subtype = card_types[1].strip()
#     print(card_supertype)
    print(card_subtype)
elif "—" not in card_type:
    card_supertype = card_type
    card_subtype = "None"   
#     print(card_supertype)
    print(card_subtype)
    
if "Legendary" in card_supertype:
    is_legendary = True
    card_supertype = re.sub("Legendary ", "", card_supertype)
else:
    is_legendary = False
print(card_supertype)    
print(is_legendary)
    
card_artist = html.find("p", class_ = "card-text-artist").text.strip().split("\n")[1].strip()
print(card_artist.encode('utf-8'))

card_current_prints = html.find("div", class_ = "prints-current").text.strip().replace("\n", "").split("              ")
card_set = card_current_prints[0]
wrds = card_set.strip().split(" ")[0:-1]
card_set = " ".join(wrds)
print(card_set)
card_rarity = card_current_prints[1].split(" · ")[1]
print(card_rarity)

card_legalities = []
for j in html.find_all("div", class_ = "card-legality-item"):
    card_legalities.append(j.text.split())
for l in card_legalities:
    if l[0] == "Standard":
        if (l[1] == "Not"):
            card_standard_legal = "Not Legal"
        elif l[1] == "Banned":
            card_standard_legal = "Banned"
        elif l[1] == "Legal":
            card_standard_legal = "Legal"
    if l[0] == "Modern":
        if (l[1] == "Not"):
            card_modern_legal = "Not Legal"
        elif l[1] == "Banned":
            card_modern_legal = "Banned"
        elif l[1] == "Legal":
            card_modern_legal = "Legal"
    if l[0] == "Commander":
        if (l[1] == "Not"):
            card_commander_legal = "Not Legal"
        elif l[1] == "Banned":
            card_commander_legal = "Banned"
        elif l[1] == "Legal":
            card_commander_legal = "Legal"
    if l[0] == "Vintage":
        if (l[1] == "Not"):
            card_vintage_legal = "Not Legal"
        elif l[1] == "Banned":
            card_vintage_legal = "Banned"
        elif l[1] == "Legal":
            card_vintage_legal = "Legal"
        elif l[1] == "Restrict.":
            card_vintage_legal = "Restrict."
    if l[0] == "Legacy":
        if (l[1] == "Not"):
            card_legacy_legal = "Not Legal"
        elif l[1] == "Banned":
            card_legacy_legal = "Banned"
        elif l[1] == "Legal":
            card_legacy_legal = "Legal"
print("sl", card_standard_legal)
print("ml", card_modern_legal)
print("cl", card_commander_legal)
print("vl", card_vintage_legal)
print("ll", card_legacy_legal)


for k in html.find_all("tr", class_ = "current"):
    if len(re.findall("\d*\.\d{2}", k.text.strip())) != 0:
        card_prices = re.findall("\d*\.\d{2}", k.text.strip())
        card_price = float(card_prices[0])
print(card_price)

l = []
for z in html.select("div[class = prints] > table[class=prints-table]"):
    l.append(z.text)
for n in l:
    sections = re.sub("  {1,}", " ", n.replace("\n", ""))
    if "Prints" in sections:
        if len(sections.split()) > 6:
            card_is_reprint = True
        else:
            card_is_reprint = False
    else:
        continue       
print("rp", card_is_reprint)

if (card_has_suspend == False) & ("Land" not in card_supertype):
    card_casting_cost = re.findall("[A-Z1-9]", card_title.strip().split("\n")[1].strip())
    try: 
        if len(card_casting_cost) == 0:
            card_cmc = 0
        else:
            generic_mana_count = int(card_casting_cost[0])
            card_cmc = generic_mana_count + len(card_casting_cost[1:])
    except ValueError:
        card_cmc = len(card_casting_cost)

    card_color_list = []
    if "W" in card_casting_cost:
        card_color_list.append("White")
    if "U" in card_casting_cost:
        card_color_list.append("Blue")
    if "B" in card_casting_cost:
        card_color_list.append("Black")
    if "R" in card_casting_cost:
        card_color_list.append("Red")
    if "G" in card_casting_cost:
        card_color_list.append("Green")
    if len(card_color_list) == 1:
        card_color = card_color_list[0]
    if len(card_color_list) > 1:
        card_color = "Gold"
    if len(card_color_list) == 0:
        card_color = "Colorless"
    print(card_cmc)
    print(card_color)

elif (card_has_suspend == False) & (("Land" in card_supertype)):
    card_cmc = 0
    card_color = "Colorless"
    print(card_cmc)

    print(card_color)

Library of Alexandria
None
Land
False
b'Mark Poole'
Arabian Nights
Uncommon
sl Not Legal
ml Not Legal
cl Banned
vl Restrict.
ll Banned
1499.99
rp True
0
Colorless


In [19]:
cols = ["names", "supertypes", "subtypes", "set", "rarity", "cmc", "color", "modern legality", "standard legality", "commander legality", "vintage legality", "legacy legality", "reprint", "artist", "price", "is_legendary"]
cards_df = pd.DataFrame(columns = cols)
for set_ in main_sets["set_id"]:
    cards_df = pd.concat([cards_df, pd.read_csv("./data/main_sets/{}_info.csv".format(set_))])
cards_df

,names,supertypes,subtypes,set,rarity,cmc,color,modern legality,standard legality,commander legality,vintage legality,legacy legality,reprint,artist,price,is_legendary
0,Aerial Assault,Sorcery,None,Core Set 2020,Common,3,White,Legal,Legal,Legal,Legal,Legal,True,b'Dan Scott',0.06,False
1,Ajani Strength of the Pride,Planeswalker,Ajani,Core Set 2020,Mythic Rare,4,White,Legal,Legal,Legal,Legal,Legal,True,b'Chris Rallis',12.42,True
2,Ancestral Blade,Artifact,Equipment,Core Set 2020,Uncommon,2,White,Legal,Legal,Legal,Legal,Legal,True,b'Scott Murphy',0.11,False
3,Angel of Vitality,Creature,Angel,Core Set 2020,Uncommon,3,White,Legal,Legal,Legal,Legal,Legal,True,b'Johannes Voss',0.26,False
4,Angelic Gift,Enchantment,Aura,Core Set 2020,Common,2,White,Legal,Legal,Legal,Legal,Legal,True,b'Josu Hernaiz',0.09,False
5,Apostle of Purifying Light,Creature,Human Cleric,Core Set 2020,Uncommon,2,White,Legal,Legal,Legal,Legal,Legal,True,b'Bastien L. Deharme',0.16,False
6,Battalion Foot Soldier,Creature,Human Soldier,Core Set 2020,Common,3,White,Legal,Legal,Legal,Legal,Legal,True,b'Cristi Balanescu',0.07,False
7,Bishop of Wings,Creature,Human Cleric,Core Set 2020,Rare,2,White,Legal,Legal,Legal,Legal,Legal,True,b'Matt Stewart',0.68,False
8,Brought Back,Instant,None,Core Set 2020,Rare,2,White,Legal,Legal,Legal,Legal,Legal,True,b'Mitchell Malloy',0.64,False
9,Cavalier of Dawn,Creature,Elemental Knight,Core Set 2020,Mythic Rare,5,White,Legal,Legal,Legal,Legal,Legal,True,b'Daarken',2.55,False


In [23]:
# cards_df = pd.read_csv(".\\data\\main_card_data.csv")
cards_df['set'] = np.where((cards_df.set == 'Ikoria: Lair of Behemoths'),'Ikoria Lair of Behemoths',cards_df.set)
cards_df['set'] = np.where((cards_df.set == "Summer Magic / Edgar"),'Summer Magic',cards_df.set)
cards_df['set'] = np.where((cards_df.set == 'Ravnica: City of Guilds'),'Ravnica City of Guilds',cards_df.set)

cards_df = cards_df[cards_df["set"] != "Fourth Edition Foreign Black Border"]
cards_df = cards_df[cards_df["set"] != "Foreign Black Border"]

cards_df = cards_df[cards_df["names"] != "Chaos Orb"]

print(len(cards_df.names.unique()))
print(len(cards_df.names))

18308
18308
25097


In [24]:
cards_df.to_csv("./data/main_card_data.csv", index = True)